In [8]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [5]:
import sys
print(sys.executable)

/Users/evanthoms/Coding/RMP-AI-Assistant/venv/bin/python


In [6]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag1", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [7]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Great professor with clear lectures, but sometimes a bit too fast-paced.'},
 {'professor': 'Prof. Mark Spencer',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Excellent at explaining complex topics and always available for help.'},
 {'professor': 'Dr. Emily Clark',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Knowledgeable but not very engaging during lectures.'},
 {'professor': 'Dr. James Roberts',
  'subject': 'Chemistry',
  'stars': 2,
  'review': "The course was challenging, but the professor didn't provide enough support."},
 {'professor': 'Prof. Sarah Miller',
  'subject': 'History',
  'stars': 4,
  'review': 'Interesting lectures, though sometimes assignments are unclear.'},
 {'professor': 'Dr. William Brown',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Engaging and enthusiastic, makes the subject fascinating.'},
 {'professor': 'Prof. Linda Davis',
  'subject': '

In [11]:
processed_data = []
client = OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

In [9]:
processed_data[0]

{'values': [-0.003443722,
  0.025151353,
  -0.056843854,
  -0.013710759,
  -0.008016112,
  0.023637911,
  -0.013582501,
  -0.005608072,
  0.03878516,
  -0.00622371,
  -0.004575597,
  0.0024673594,
  -0.011414944,
  -0.016147656,
  0.0125051355,
  -0.044941533,
  -0.02604916,
  -0.03932384,
  0.0032192708,
  0.0071952627,
  -0.009279451,
  -0.036579125,
  0.016263088,
  0.010542791,
  -0.059511617,
  -0.0023022275,
  0.03257748,
  0.0036521407,
  0.054022186,
  0.018841071,
  0.05117486,
  -0.011793304,
  0.025125703,
  -0.040349904,
  -0.052380484,
  0.043068968,
  0.009516729,
  0.025459172,
  0.021572962,
  0.03403962,
  0.008830549,
  0.019918436,
  -0.021034278,
  -0.0075543844,
  0.07151655,
  0.0049956413,
  -0.0022605436,
  -0.014403351,
  0.017981743,
  0.027370214,
  -0.047968417,
  0.027985852,
  0.042838104,
  0.067155786,
  -0.028473232,
  0.011267448,
  0.032038797,
  0.0074710166,
  0.0361687,
  -0.008016112,
  0.08295714,
  -0.033321377,
  0.019136064,
  -0.02911452,
  -

In [13]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [14]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}